In [1]:
import sys
import pickle
sys.path.insert(0,'./')
import pyccl as ccl
# from importlib import reload

In [2]:
s=[0,4]
4 in s

True

In [3]:
from distributed import LocalCluster
from dask.distributed import Client  # we already had this above
#http://distributed.readthedocs.io/en/latest/_modules/distributed/worker.html
LC=LocalCluster(n_workers=1,processes=False,memory_limit='50gb',threads_per_worker=10,memory_spill_fraction=.99,
               memory_monitor_interval='2000ms')
client=Client(LC)

/usr/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [4]:
client

Client Scheduler: inproc://136.152.250.206/2976/1 Dashboard: http://localhost:36495/status,Cluster Workers: 1 Cores: 10 Memory: 50.00 GB


In [5]:
from cov_3X2 import *
from lsst_utils import *
# from fisher import *

In [6]:
#only for python3
import importlib
reload=importlib.reload

# Lensing X Lensing Correlation function covariance

## Multiple redshift bins (Tomography)

In [7]:
import cov_utils
reload(cov_utils)
from cov_utils import *

import window_utils
reload(window_utils)
from window_utils import *

In [8]:
import cov_3X2
reload(cov_3X2)
from cov_3X2 import *
from wigner_transform import *

In [9]:
import hankel_transform
from astropy.io import fits
reload(hankel_transform)
from hankel_transform import *


In [10]:
cosmo_params={'h': 0.8433,
 'Omb': 0.062,
 'Omd': 0.2837-0.062,
 's8': 0.795431,
 'Om': 0.2837,
 'As': 1.685e-09,
'Ase9': 1.685,
 'mnu': 0.0,
 'Omk': 0.0,
 'tau': 0.06,
 'ns': 0.96859,
 'w': -1,
 'wa': 0,
 'Oml': 1.-0.2837}
PS=Power_Spectra(cosmo_params=cosmo_params)
k,pk,s8=PS.camb_pk(z=[0],cosmo_params=cosmo_params,return_s8=True)
s8

0.7971166457654738

In [11]:
mnu=cosmo_h.m_nu*0
cosmo=cosmo.clone(m_nu=mnu,Ob0=cosmo_params['Omb'],Om0=cosmo_params['Om'])
cosmo_h=cosmo.clone(H0=100) #,m_nu=mnu,Ob0=cosmo_params['Omb'],Om0=cosmo_params['Om'])
ccl_cosmo = ccl.Cosmology(Omega_c=cosmo_params['Omd'], Omega_b=cosmo_params['Omb'], 
                      h=cosmo_params['h'], A_s=cosmo_params['As'], n_s=cosmo_params['ns'],)#m_nu=[0,0,0.06])

In [12]:
cov_des=np.genfromtxt('../covariance_testing/data/DES-Y3//cosmolike_cov_Y3ish/cov_y3_mcal_emu_final.txt')
covG_des=np.genfromtxt('../covariance_testing/data/DES-Y3/cosmolike_cov_Y3ish/cov_y3_mcal_emu_final.txt') 
# covNG_shear=np.genfromtxt('/home/deep/repos/DESC_SRD/cov/Y10_shear_shear_covNG',) 

l_min0=20
l_max0=15000
nl0=20
l_bins=np.logspace(np.log10(l_min0),np.log10(l_max0),nl0+1)
lb=0.5*(l_bins[1:]+l_bins[:-1])

lb15=np.tile(lb,15)
x=lb15<3000
covG_shear=covG_des[:,2].reshape(900,900)[:400,:400] #10 X 2 X 20
cov_shear=cov_des[:,2].reshape(900,900)[:400,:400]

covG_ggl=covG_des[:,2].reshape(900,900)[400:800,400:800] #10 X 2 X 20
cov_ggl=cov_des[:,2].reshape(900,900)[400:800,400:800]

covG_gg=covG_des[:,2].reshape(900,900)[800:900,800:900] #10 X 2 X 20
cov_gg=cov_des[:,2].reshape(900,900)[800:900,800:900]


cov_des=cov_des[:,2].reshape(900,900)
covG_des=covG_des[:,2].reshape(900,900)
# covNG_shear=covNG_shear[x].T[x].T

x=lb<3000
lb=lb[x]
l_bins=l_bins[:16]

In [13]:
sigma_gamma=0.3944/sqrt(2.)  #*2**0.25

In [14]:
window_lmax=30

In [15]:
l_max=4e4
l_maxW=6e4
l_min=2
# l0=np.arange(l_min,l_maxW)   #np.arange(l_min,l_max)
l0=np.logspace(np.log10(l_min),np.log10(l_maxW),6.e3)
lw=np.arange(window_lmax)
l0=np.sort(np.append(l0,lw))
# l0=np.sort(np.append(l0,np.logspace(np.log10(l_maxW),np.log10(l_maxW*10),1.e3)))
l0=np.unique(np.int32(l0))   #np.arange(l_min,l_max)
# l0=np.unique(np.int32(np.logspace(np.log10(l_min),np.log10(l_maxW),4.e3)))   #np.arange(l_min,l_max)

In [16]:
#Setup lens redshift bins
source_f='../covariance_testing/data/DES-Y3//cosmolike_cov_Y3ish/lens.nz'
zmin=0
zmax=3.5
f_sky=5000./(4*np.pi/d2r**2)

nl_bins=4  # Number of tomographic bins
zl_bins=DES_lens_bins(fname=source_f,l=[1],f_sky=f_sky)

./cov_utils.py:60: RuntimeWarning: invalid value encountered in true_divide
  Win0=2*jn(1,l_th)/l_th
./angular_power_spectra.py:23: RuntimeWarning: divide by zero encountered in true_divide
  self.cl_f=(l+0.5)**2/(l*(l+1.)) # cl correction from Kilbinger+ 2017


corr_indxs {('shear', 'shear'): [], ('galaxy', 'galaxy'): [(0, 0), (1, 1), (2, 2), (3, 3), (4, 4)], ('kappa', 'kappa'): []}
Win gen: step size 768
cl dict done


In [17]:
#Setup source redshift bins
source_f='../covariance_testing/data/DES-Y3//cosmolike_cov_Y3ish/source.nz'
zmin=0
zmax=3.5


nbins=4  # Number of tomographic bins
zs_bins=DES_bins(fname=source_f,sigma_gamma=sigma_gamma,l=[1],f_sky=f_sky)

corr_indxs {('shear', 'shear'): [], ('galaxy', 'galaxy'): [(0, 0), (1, 1), (2, 2), (3, 3), (4, 4)], ('kappa', 'kappa'): []}
Win gen: step size 768
cl dict done


In [18]:
# f_sky=4182./(4*np.pi/d2r**2)

In [19]:
f_sky*4*np.pi

1.5230870989335428

In [20]:
do_xi=True
bin_xi=True
do_cov=True
bin_cl=False
th_min=2.5/60
th_max=250./60
n_th_bins=20
th_bins=np.logspace(np.log10(th_min),np.log10(th_max),n_th_bins+1)
th=np.logspace(np.log10(th_min*0.98),np.log10(1),n_th_bins*30)
th2=np.linspace(1,th_max*1.02,n_th_bins*30)
# th2=np.logspace(np.log10(1),np.log10(th_max),60*6)
th=np.unique(np.sort(np.append(th,th2)))
thb=np.sqrt(th_bins[1:]*th_bins[:-1])

In [21]:
#Hankel Transform setup
prune_theta=0
HT_kwargs={'l_min':l_min,  'l_max':l_max,
                        'theta_min':th_min*d2r*.9, 'theta_max':th_max*d2r,
                        'n_zeros':40000, 'prune_theta':prune_theta, 'm1_m2':[(2,2),(2,-2),(0,2),(0,0)]}
# %time HT=hankel_transform(**HT_kwargs)
# HT_kwargs2={'l_min':l_min,  'l_max':l_max,
#                         'theta_min':th_min*d2r*.9, 'theta_max':th_max*d2r,
#                         'n_zeros':40000, 'prune_theta':2, 'm1_m2':[(2,-2)]}#,(2,-2)]}

In [22]:
use_window=False
do_sample_variance=True
SSV_cov=False
tidal_SSV=True


In [23]:
import cov_utils
reload(cov_utils)
from cov_utils import *
import cov_3X2
reload(cov_3X2)
from cov_3X2 import *


In [24]:
power_spectra_kwargs={'cosmo':cosmo,'cosmo_params':cosmo_params,'pk_func':'camb_pk_too_many_z'}

In [45]:
corr_ggl=('shear','galaxy')
corr_gg=('galaxy','galaxy')
corr_ll=('shear','shear')
m1m2=(0,2)
m1m2_gg=(0,0)

corrs=[corr_ll,corr_ggl,corr_gg]

In [26]:
store_win=True

In [27]:
#### Wigner Transform setup... Using high ell to test convergence
WT_L_kwargs={'l': l0,'theta': th*d2r,'m1_m2':[(2,2),(2,-2),(0,2),(2,0),(0,0)]}
%time WT_L=wigner_transform(**WT_L_kwargs)

./wigner_functions.py:38: RuntimeWarning: invalid value encountered in true_divide
  d_mat/=np.sqrt(binom(k+b,b))
./wigner_functions.py:38: RuntimeWarning: divide by zero encountered in true_divide
  d_mat/=np.sqrt(binom(k+b,b))
./wigner_functions.py:38: RuntimeWarning: divide by zero encountered in true_divide
  d_mat/=np.sqrt(binom(k+b,b))


CPU times: user 817 ms, sys: 953 ms, total: 1.77 s
Wall time: 28.1 s


In [ ]:
XI_W_L= cov_3X2(zs_bins=zs_bins,l=l0,do_cov=do_cov,bin_cl=bin_cl,l_bins=l_bins,zg_bins=zl_bins,
                do_sample_variance=do_sample_variance,use_window=use_window,
                power_spectra_kwargs=power_spectra_kwargs,corrs=corrs,
                store_win=store_win,window_lmax=window_lmax,
               SSV_cov=SSV_cov,tidal_SSV_cov=tidal_SSV,do_xi=True,HT=WT_L,f_sky=f_sky,
               bin_xi=bin_xi,theta_bins=th_bins,
               xi_win=True)#ns=np.inf)

corr_indxs {('shear', 'shear'): [(0, 0), (0, 1), (0, 2), (0, 3), (1, 1), (1, 2), (1, 3), (2, 2), (2, 3), (3, 3)], ('galaxy', 'galaxy'): [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 1), (1, 2), (1, 3), (1, 4), (2, 2), (2, 3), (2, 4), (3, 3), (3, 4), (4, 4)], ('kappa', 'kappa'): [], ('shear', 'galaxy'): [(0, 0), (1, 0), (2, 0), (3, 0), (0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2), (2, 2), (3, 2), (0, 3), (1, 3), (2, 3), (3, 3), (0, 4), (1, 4), (2, 4), (3, 4)], ('galaxy', 'shear'): [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (0, 3), (1, 3), (2, 3), (3, 3), (4, 3)]}
Win gen: step size 1874
setting windows <Client: scheduler='inproc://136.152.250.206/2976/1' processes=1 cores=10>


In [ ]:
gg_i=[(i,i) for i in np.arange(zl_bins['n_bins'])]
XI_W_L.stack_indxs[corr_gg]=gg_i
# XI_W_L.stack_indxs

In [ ]:
bi=(0,0)

In [ ]:
XI_W_L.Win.Win['cov'][corr_ll+corr_ll][bi+bi]['xi']

In [ ]:
XI_W_L.Win.Win['cl'][corr_ll][bi]['xi']

In [ ]:
XI_W_L.Win.window_l

In [ ]:
#### Wigner Transform setup... Using high ell to test convergence
WT_L_kwargs1={'l': XI_W_L.Win.window_l,'theta': th*d2r,'m1_m2':[(2,2),(2,-2),(0,2),(2,0),(0,0)]}
%time WT_L1=wigner_transform(**WT_L_kwargs1)

In [ ]:
XI_W_L.Win.Win['cl'][corr_ll][bi].keys()

In [ ]:
WT_L1.projected_correlation(cl=XI_W_L.Win.Win['cl'][corr_ll][bi][12]['cl'],l_cl=XI_W_L.Win.window_l,m1_m2=(0,0))

In [ ]:
plot(XI_W_L.Win.window_l,XI_W_L.Win.Win['cl'][corr_ll][bi][12]['cl'])
# xscale('log')
yscale('log')

In [ ]:
# %time xiHG=XI_H.xi_tomo()
# %time xiHG2=XI_H2.xi_tomo()
%time xiWG_L=XI_W_L.xi_tomo()

In [ ]:
# plot(thb,np.diag(cov_shear/xiH['cov'])[:20])
# plot(thb,xiH['xi'][:20])
ii=0
i=ii*20
j=i+20
ii=0
sn=np.diag(cov_shear)[0]/thb**2*thb[0]**2
cc=xiWG_L['cov'][('shear','shear','shear','shear')][(2,2,2,2)][(ii,ii,ii,ii)].compute()
sn2=np.diag(cc['final'])[0]/thb**2*thb[0]**2
sn2*=1.#4.e5
plot(thb,sn,'r-')
plot(thb,sn2,'ro')
plot(thb,np.diag(cc['final']),'bo')
# plot(thb,np.diag(xiW_L['cov'])[i:j],'bo')
plot(thb,np.diag(cov_shear)[i:j],label='Cosmolike')
# plot(thb,np.diag(cov_shear)[0]/thb**2*thb[0]**2)
xscale('log')
yscale('log')
xlabel(r'$\theta$')
ylabel(r'$\delta \xi_+$')
legend()

In [ ]:
XI_W_L.Win.Win['cl'].keys()

In [ ]:
sn2

In [ ]:
# plot(thb,np.diag(cov_shear/xiH['cov'])[:20])
# plot(thb,xiH['xi'][:20])
ii=19
i=ii*20
j=i+20
ii=3
sn=0#np.diag(cov_shear)[0]/thb**2*thb[0]**2
cc=xiWG_L['cov'][('shear','shear','shear','shear')][(2,2,2,2)][(ii,ii,ii,ii)].compute()

plot(thb,np.diag(cc['final']),'bo')
# plot(thb,np.diag(xiW_L['cov'])[i:j],'bo')
plot(thb,np.diag(cov_shear)[i:j],label='Cosmolike')
# plot(thb,np.diag(cov_shear)[0]/thb**2*thb[0]**2)
xscale('log')
yscale('log')
xlabel(r'$\theta$')
ylabel(r'$\delta \xi_+$')
legend()

In [ ]:
# plot(thb,np.diag(cov_shear/xiH['cov'])[:20])
# plot(thb,xiH['xi'][:20])
ii=0
i=ii*20
ii2=1
j=ii2*20

cc2=cov_ggl[i:i+20,j:j+20]
sn=np.diag(cc2)[0]/thb**2*thb[0]**2

cc=xiWG_L['cov'][('shear','galaxy','shear','galaxy')][(2,0,2,0)][(0,0,1,0)].compute()
cc3=xiWG_L['cov'][('shear','galaxy','shear','galaxy')][(2,0,2,0)][(0,0,0,0)].compute()
# cc=xiWG_L['cov'][('galaxy','shear','galaxy','shear')][(0,2,0,2)][(0,0,0,0)].compute()

# plot(thb,np.diag(xiW_L['cov'])[i:j],'bo')
plot(thb,np.diag(cc['final']),'bo')
# plot(thb,sn,':')
plot(thb,np.diag(cc2))
# plot(thb,np.diag(cc3['final']),'ro')
# plot(thb,np.diag(cov_shear)[0]/thb**2*thb[0]**2)
xscale('log')
yscale('log')

In [ ]:
# plot(thb,np.diag(cov_shear/xiH['cov'])[:20])
# plot(thb,xiH['xi'][:20])
ii=0
i=ii*20
ii2=0
j=ii2*20

cc2=cov_ggl[i:i+20,j:j+20]
sn=np.diag(cc2)[0]/thb**2*thb[0]**2

cc=xiWG_L['cov'][('shear','galaxy','shear','galaxy')][(2,0,2,0)][(0,0,0,0)].compute()


plot(thb,np.diag(cc['final']),'bo')
# plot(thb,sn,':')
plot(thb,np.diag(cov_ggl)[i:i+20])
# plot(thb,np.diag(cc3['final']),'ro')
# plot(thb,np.diag(cov_shear)[0]/thb**2*thb[0]**2)
xscale('log')
yscale('log')

In [ ]:
plot(np.tile(thb,20),np.diag(cov_ggl),'bo')
xscale('log')
yscale('log')

In [ ]:
%time xiW_L=xiWG_L['stack'].compute()

In [ ]:
f=1
# cov_xiH_inv=np.linalg.inv(xiH['cov']*f)
# print (np.sqrt(np.dot(xiH['xi'],np.dot(cov_xiH_inv,xiH['xi']))))

cov_xiWL_inv=np.linalg.inv(xiW_L['cov']*f)
print (np.sqrt(np.dot(xiW_L['xi'],np.dot(cov_xiWL_inv,xiW_L['xi']))))

In [ ]:
cc=XI_W_L.cov_utils
corrW=cc.corr_matrix(xiW_L['cov'])
pcolor(corrW,vmin=-1,vmax=1,cmap='seismic')
# xlabel(r'$\theta$ (degrees)')
# ylabel(r'$\theta$ (degrees)')
title('Wigner')
# hlines(60,0,120)
# vlines(60,0,120)
xticks([])
yticks([])


colorbar()
show()
# corrL=cc.corr_matrix(clL['cov'])
# pcolor(corrL,vmin=-1,vmax=1,cmap='seismic')

In [ ]:
cc=XI_W_L.cov_utils
corr_shear2=cc.corr_matrix(covG_ggl)
pcolor(corr_shear2,vmin=-1,vmax=1,cmap='seismic')
# xlabel(r'$\theta$ (degrees)')
# ylabel(r'$\theta$ (degrees)')
title('Cosmolike')
# hlines(60,0,120)
# vlines(60,0,120)
xticks([])
yticks([])


colorbar()
show()
# corrL=cc.corr_matrix(clL['cov'])
# pcolor(corrL,vmin=-1,vmax=1,cmap='seismic')

In [ ]:
cc=XI_W_L.cov_utils
corr_shear=cc.corr_matrix(covG_des)
pcolor(corr_shear,vmin=-1,vmax=1,cmap='seismic')
# xlabel(r'$\theta$ (degrees)')
# ylabel(r'$\theta$ (degrees)')
title('Cosmolike')
# hlines(60,0,120)
# vlines(60,0,120)
xticks([])
yticks([])


colorbar()
show()
# corrL=cc.corr_matrix(clL['cov'])
# pcolor(corrL,vmin=-1,vmax=1,cmap='seismic')

In [ ]:
pcolor(corr_shear-corrW,vmin=-1,vmax=1,cmap='seismic')
colorbar()
xticks([])
yticks([])

In [ ]:
pcolor(cov_des/xiW_L['cov']-1,cmap='seismic',vmin=-1,vmax=1,)
colorbar()

In [ ]:
plot(thb,XI_W_L.xi_bin_utils[(2,2)]['norm']/thb**2)
xscale('log')
# yscale('log')

In [ ]:
cov_test=np.genfromtxt('shape_noise_cov.dat')
cov_test2=np.genfromtxt('shape_noise_diag.dat')

In [ ]:
# plot(thb,np.diag(cov_shear/xiH['cov'])[:20])
# plot(thb,xiH['xi'][:20])
ii=0
i=ii*20
j=i+20
sn=np.diag(cov_shear)[0]/thb**2*thb[0]**2
# NN=np.diag(xiH['cov'])[i:j]-sn/2.
# plot(thb,np.diag(xiH['cov'])[i:j],'--')
# plot(thb,np.diag(xiH2['cov'])[i:j],'--')
plot(thb,np.diag(xiW_L['cov'])[i:j],'bo')
# plot(thb,np.diag(cov_test),':')
# plot(thb,cov_test2,':')
# plot(thb,sn,':')
plot(thb,np.diag(cov_shear)[i:j])
# plot(thb,np.diag(cov_shear)[0]/thb**2*thb[0]**2)
xscale('log')
yscale('log')

In [ ]:
# plot(thb,np.diag(cov_shear/xiH['cov'])[:20])
# plot(thb,xiH['xi'][:20])
ii=1
i=ii*20
j=i+20

cc=xiWG_L['cov'][('galaxy','galaxy','galaxy','galaxy')][(0,0,0,0)][(ii,ii,ii,ii)].compute()

sn=np.diag(cov_gg)[0]/thb**2*thb[0]**2
# plot(thb,np.diag(xiW_L['cov'])[i:j],'bo')
plot(thb,np.diag(cc['final']),'bo')
# plot(thb,sn,':')
plot(thb,np.diag(cov_gg)[i:j])
# plot(thb,np.diag(cov_shear)[0]/thb**2*thb[0]**2)
xscale('log')
yscale('log')

In [ ]:
# plot(thb,np.diag(cov_shear/xiH['cov'])[:20])
# plot(thb,xiH['xi'][:20])
ii=4
i=ii*20
ii2=0
j=ii2*20

cc2=cov_ggl[i:i+20,j:j+20]
sn=np.diag(cc2)[0]/thb**2*thb[0]**2

cc=xiWG_L['cov'][('shear','galaxy','shear','galaxy')][(2,0,2,0)][(0,0,0,1)].compute()
# cc=xiWG_L['cov'][('galaxy','shear','galaxy','shear')][(0,2,0,2)][(0,0,0,0)].compute()

# plot(thb,np.diag(xiW_L['cov'])[i:j],'bo')
plot(thb,np.diag(cc['final']),'bo')
# plot(thb,sn,':')
plot(thb,np.diag(cc2))
# plot(thb,np.diag(cov_shear)[0]/thb**2*thb[0]**2)
xscale('log')
yscale('log')

In [ ]:
XI_W_L.galaxy_utils.nz_F

In [ ]:
corrs_t=('shear','galaxy','shear','galaxy')
indxs_t=(0,0,0,0)
XI_W_L.cov_utils.get_SN(XI_W_L.SN,corrs_t,indxs_t)

In [ ]:
# plot(thb,np.diag(cov_shear/xiH['cov'])[:20])
# plot(thb,xiH['xi'][:20])
ii=0
i=ii*20
ii2=1
j=ii2*20

cc2=cov_ggl[i:i+20,j:j+20]
sn=np.diag(cc2)[0]/thb**2*thb[0]**2

cc=xiWG_L['cov'][('shear','galaxy','shear','galaxy')][(2,0,2,0)][(0,0,1,0)].compute()
cc3=xiWG_L['cov'][('shear','galaxy','shear','galaxy')][(2,0,2,0)][(0,0,0,0)].compute()
# cc=xiWG_L['cov'][('galaxy','shear','galaxy','shear')][(0,2,0,2)][(0,0,0,0)].compute()

# plot(thb,np.diag(xiW_L['cov'])[i:j],'bo')
plot(thb,np.diag(cc['final']),'bo')
# plot(thb,sn,':')
plot(thb,np.diag(cc2))
# plot(thb,np.diag(cc3['final']),'ro')
# plot(thb,np.diag(cov_shear)[0]/thb**2*thb[0]**2)
xscale('log')
yscale('log')

In [ ]:
plot(thb,np.diag(xiW_L['cov'])[i:j]-cov_test2,'--')
plot(thb,np.diag(xiH['cov'])[i:j]-cov_test2,'--')
plot(thb,(np.diag(cov_shear)[i:j]-cov_test2)*1.2)

xscale('log')
yscale('log')

In [ ]:
plot(thb,np.diag(xiW_L['cov'])[i:j],':')
plot(thb,np.diag(cov_test),':')
plot(thb,cov_test2,':')

xscale('log')
yscale('log')

In [ ]:
plot(thb,np.diag(cov_shear/xiH['cov'])[:20])
plot(thb,np.diag(cov_shear/xiW_L['cov'])[:20])
xscale('log')
# yscale('log')

In [ ]:
d2r/60

In [ ]:
XI_H.z_bins['shear'][0]['nz']

In [ ]:
XI_H.SN

In [ ]:
ns=XI_H.z_bins['shear'][0]['nz']*3600/d2r**2
sigma_gamma**2/ns

In [ ]:
XI_H.z_bins['shear'][0]['SN']

In [ ]:
plot(np.diag(xiW_L['cov']/xiH['cov'])[:20])
ylim(0.9,1.1)

In [ ]:
crash

In [ ]:
for i in np.arange(nbins):
    for j in np.arange(i,nbins):
        lens1 = ccl.ClTracerLensing(ccl_cosmo, False, z=zs_bins[i]['z'], n=zs_bins[i]['pz'])
        lens2 = ccl.ClTracerLensing(ccl_cosmo, False, z=zs_bins[j]['z'], n=zs_bins[j]['pz'])
        cls = ccl.angular_cl(ccl_cosmo, lens1, lens2, CL.l)
        plot(CL.l,clG['cl']['cl'][corr][(i,j)].compute()/cls,label=(i,j))
        xscale('log')
#         yscale('log')
        xlabel(r'$\ell$')
        ylabel(r'$C_\ell$')
        hlines(1,2,1000)
        legend()
        ylim(0.9,1.1)
    show()

In [ ]:
m1m2=(2,2)
cp=['r','b','g','y']#sns.color_pallete()
for i in np.arange(nbins):
    for j in np.arange(i,nbins):
        lens1 = ccl.ClTracerLensing(ccl_cosmo, False, z=zs_bins[i]['z'], n=zs_bins[i]['pz'])
        lens2 = ccl.ClTracerLensing(ccl_cosmo, False, z=zs_bins[j]['z'], n=zs_bins[j]['pz'])
        cls = ccl.angular_cl(ccl_cosmo, lens1, lens2, CL.l)
        cls_b=CL.binning.bin_1d(xi=cls,bin_utils=CL.cl_bin_utils)
        plot(lb,cls_b,color=cp[j],ls=':')
        
    xscale('log')
    yscale('log')
    xlabel(r'$\ell$')
    ylabel(r'$C_\ell$')
    legend(ncol=3)
    show()

In [ ]:
def func(x):
    return x+1

In [ ]:
y=dask.delayed(func)(10)

In [ ]:
y.compute()

In [ ]:
y